In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd

! pip install datasets transformers sacrebleu
! pip install sentencepiece

from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

from transformers import AutoTokenizer

from transformers import pipeline


     |████████████████████████████████| 262 kB 4.0 MB/s 
     |████████████████████████████████| 2.5 MB 5.1 MB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 
     |████████████████████████████████| 118 kB 17.7 MB/s 
     |████████████████████████████████| 43 kB 1.1 MB/s 
     |████████████████████████████████| 243 kB 18.9 MB/s 
     |████████████████████████████████| 895 kB 18.4 MB/s 
     |████████████████████████████████| 3.3 MB 21.7 MB/s 
     |████████████████████████████████| 1.2 MB 3.9 MB/s 


In [3]:
translator_tr_en = pipeline("translation",model="/content/drive/MyDrive/News article summarizer/Tr_En")
translator_ru_en = pipeline("translation",model="/content/drive/MyDrive/News article summarizer/Ru_En")
summarizer = pipeline("summarization",model="/content/drive/MyDrive/News article summarizer/summary")
headline = pipeline("summarization", model="/content/drive/MyDrive/News article summarizer/Heading")

In [4]:
def split_string(text, sep=" "):
    words = text.split()
    res, part, others = [], words[0], words[1:]
    for word in others:
        if len(sep)+len(word) > 512-len(part):
            res.append(part)
            part = word
        else:
            part += sep+word
    if part:
        res.append(part)
    return res

In [5]:
! pip install langdetect
from langdetect import detect

input_article= input("Please enter an article : ")

detection=detect(input_article)

if detection=="en":
      paragraph=input_article
      
      
elif detection=="ru":
      output=[]
      a=input_article.split(".")
      for i in a:
        if len(i)<=512:
            z= translator_ru_en(i, max_length=100000)
            for x in z:
                x['translation_text']=x['translation_text'].replace(".","")
                output.append(x['translation_text'])

        else:
            y=split_string(i)
            output1=""
            for x in y:
              b=translator_ru_en(x, max_length=1000000)
              for x in b:
                   x['translation_text']=x['translation_text'].replace(".","")
                   output1= output1+" "+x['translation_text']
            output.append(output1)
      output.pop()
      paragraph=""
      for x in output:
        paragraph= paragraph+ x +". "
    
  
elif detection=="tr":
      output=[]
      a=input_article.split(".")
      for i in a:
        if len(i)<=512:
            z= translator_tr_en(i, max_length=1000000)
            for x in z:
                 x['translation_text']=x['translation_text'].replace(".","")
                 output.append(x['translation_text'])
 
        else:
            y=split_string(i)
            output1=""
            for x in y:
               b=translator_tr_en(x, max_length=100000)
               for x in b:
                      x['translation_text']=x['translation_text'].replace(".","")
                      output1= output1+" "+x['translation_text']
            output.append(output1)
      output.pop()
      paragraph=""
      for x in output:
        paragraph= paragraph+ x +". "

else:
      print("unsupported")

     |████████████████████████████████| 981 kB 3.0 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993241 sha256=83b7b5c8009406eb66460221f0dfead48a8308d11df3deb77e398a2c224f06cb
  Stored in directory: /root/.cache/pip/wheels/c5/96/8a/f90c59ed25d75e50a8c10a1b1c2d4c402e4dacfa87f3aff36a
Successfully built langdetect
Please enter an article : There could be supersonic passenger planes this decade. Supersonic means faster than the speed of sound, which is 1,234kph. United Airlines will buy 15 supersonic jets. The planes will fly up to 88 business class passengers and have a top speed of 2,099kph. Most modern airplanes fly at around 1,000kph. The new planes will cut the time it takes to fly from London to New York from around 7 1/2 hours to less than four hours.       The United Airlines jets will be the first supersonic jets to make commercial flights since 2003. The cost of the tickets will be about the same as regular business class tickets. Previousl

In [6]:
output=[]
a=paragraph.split(".")

x=len(a)
y=int((x-x%5)/5)

d=[]
for i in range(y):
   b=[]
   for i in range(5):
      b.append(a[i])
   c=""
   for i in b:
      c=c+i+"."
   d.append(c)
   del a[0:5]
b=[]
for i in range((x%5)):
    b.append(a[i])
c=""
for i in b:
    c=c+i+"."
d.append(c)

m=[]
paragraph1=""
for x in d:
  z = summarizer(x)
  m.append(z)
m.pop()
for i in m:
  for x in i:
    paragraph1=paragraph1+x['summary_text']
summary="".join( paragraph1.splitlines())
b=headline(paragraph)
for i in b:
  heading=i['summary_text']

Your max_length is set to 128, but you input_length is only 77. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
Your max_length is set to 128, but you input_length is only 95. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 128, but you input_length is only 38. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=

In [7]:
print(heading+'\n')
print(summary)

United Airlines to buy 15 supersonic planes to fly faster than the speed of sound - The New York Times - Breitbart

United Airlines will buy 15 supersonic jets .The planes will fly up to 88 business class passengers .Most modern airplanes fly at around 1,000kph .The United Airlines jets will be the first supersonic jets to make commercial flights since 2003 .The cost of the tickets will be about the same as regular business class tickets .


In [8]:
qa_pipeline = pipeline(
    
    "question-answering",
    model ="bert-large-uncased-whole-word-masking-finetuned-squad",
    tokenizer="bert-large-uncased-whole-word-masking-finetuned-squad"
    
)

In [9]:
a=input('Do you have any questions regarding input article? (yes/no)')
while (a=='yes'):
  question=input('Enter your question: ')
  predictions = qa_pipeline({
                'context' : paragraph,
                'question' : question
  })
  
  print(predictions['answer'])
  a=input('Do you have any questions regarding input article? (yes/no)')
  if (a=='no'):
     print("Thank you.")

Do you have any questions regarding input article? (yes/no)yes
Enter your question: what is the cost of ticket
about the same as regular business class tickets
Do you have any questions regarding input article? (yes/no)yes
Enter your question: speed of plane
Supersonic means faster than the speed of sound
Do you have any questions regarding input article? (yes/no)yes
Enter your question: airline will buy how many jets
15 supersonic jets
Do you have any questions regarding input article? (yes/no)no
Thank you.


In [10]:
qa_pipeline = pipeline(
    
    "question-answering",
    model ="google/bigbird-base-trivia-itc",
    tokenizer="google/bigbird-base-trivia-itc"
    
)

In [12]:
a=input('Do you have any questions regarding input article? (yes/no)')
while (a=='yes'):
  question=input('Enter your question: ')
  predictions = qa_pipeline({
                'context' : paragraph,
                'question' : question
  })
  
  print(predictions['answer'])
  a=input('Do you have any questions regarding input article? (yes/no)')
  if (a=='no'):
     print("Thank you.")

Do you have any questions regarding input article? (yes/no)yes
Enter your question: speed of plane
 1,234kph.
Do you have any questions regarding input article? (yes/no)yes
Enter your question: cost of tickets
 same
Do you have any questions regarding input article? (yes/no)no
Thank you.


In [13]:
qa_pipeline = pipeline(
    
    "question-answering",
    model ="csarron/bert-base-uncased-squad-v1",
    tokenizer="csarron/bert-base-uncased-squad-v1"
    
)

In [ ]:
a=input('Do you have any questions regarding input article? (yes/no)')
while (a=='yes'):
  question=input('Enter your question: ')
  predictions = qa_pipeline({
                'context' : paragraph,
                'question' : question
  })
  
  print(predictions['answer'])
  a=input('Do you have any questions regarding input article? (yes/no)')
  if (a=='no'):
     print("Thank you.")

Do you have any questions regarding input article? (yes/no)yes
Enter your question: how many jets does airline buy
15
